In [2]:
import os
os.chdir('/Users/layne/Desktop/aiqc/aiqc/data')
os.getcwd()

'/Users/layne/Desktop/aiqc/aiqc/data'

In [3]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import *

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import History

/Users/layne/.pyenv/versions/3.7.6/envs/jupyterlab/lib/python3.7/site-packages/pandas/compat/__init__.py:97: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [5]:
df = pd.read_parquet("dtype_testing.parquet")

In [9]:
df.dtypes

doom_float64          float64
radiance_float32      float32
temperature_float     float64
weekday_object         object
hour_object            string
moonday_int64           int64
month_int32             int32
color_uint8             uint8
shape_int               int64
kingdom_category     category
dtype: object

In [22]:
# ### ===================== DATASET SONAR (BINARY) =====================
# https://machinelearningmastery.com/binary-classification-tutorial-with-the-keras-deep-learning-library/

source = pd.read_csv("iris.tsv", sep='\t')
dataset = source.values

X = dataset[:,0:4]
Y = dataset[:,4]
Y = np.reshape(Y, (-1, 1))

encoder_labels = OneHotEncoder(sparse=False)
encoder_features = StandardScaler()

train_x, test_x, train_y, test_y = train_test_split(X, Y, test_size=0.20)

encoder_features = encoder_features.fit(train_x)
train_x = encoder_features.transform(train_x)
test_x = encoder_features.transform(test_x)

encoder_labels = encoder_labels.fit(train_y)
train_y = encoder_labels.transform(train_y)
test_y = encoder_labels.transform(test_y)

In [23]:
model = Sequential()
model.add(Dense(9, activation='relu', input_shape=(4,), kernel_initializer='he_uniform'))
model.add(Dropout(0.20))
model.add(Dense(9, activation='relu', kernel_initializer='he_uniform'))
model.add(Dense(3, activation='softmax'))
#model.compile(optimizer="RMSprop", loss='binary_crossentropy', metrics=['accuracy'])
model.compile(optimizer="adamax", loss='categorical_crossentropy')

model.fit(
    train_x, train_y
    , validation_data=(test_x, test_y)
    , verbose=0
    , batch_size=3, epochs=100
    , callbacks=[History()]
)
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 9)                 45        
_________________________________________________________________
dropout_1 (Dropout)          (None, 9)                 0         
_________________________________________________________________
dense_4 (Dense)              (None, 9)                 90        
_________________________________________________________________
dense_5 (Dense)              (None, 3)                 30        
Total params: 165
Trainable params: 165
Non-trainable params: 0
_________________________________________________________________
None


In [4]:
model.predict_classes(train_x)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


array([[1],
       [0],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [0],
       [1],
       [1],
       [1],
       [0],
       [1],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [1],
       [0],
    

In [5]:
probs = model.predict(train_x)

In [31]:
preds = np.argmax(model.predict(train_x), axis=-1)

In [24]:
metrics = model.evaluate(test_x, test_y, verbose=0)

In [11]:
str(type(metrics)) == "<class 'list'>"

True

In [12]:
isinstance(metrics, list)

True

---

In [6]:
accuracy_score(train_y, preds)

1.0

In [7]:
precision_score(train_y, preds, average='binary')

1.0

In [8]:
recall_score(train_y, preds, average='binary')

1.0

In [9]:
f1_score(train_y, preds, average='binary')

1.0

In [10]:
roc_auc_score(train_y, probs, average='micro')

1.0

---

In [75]:
roc_curve(train_y, probs)

(array([0., 0., 0., 0., 0., 0., 0., 1.]),
 array([0.        , 0.92222222, 0.93333333, 0.95555556, 0.96666667,
        0.98888889, 1.        , 1.        ]),
 array([2.0000000e+00, 1.0000000e+00, 9.9999988e-01, 9.9999952e-01,
        9.9999946e-01, 9.9999934e-01, 9.9999845e-01, 9.3206930e-26],
       dtype=float32))

In [76]:
precision_recall_curve(train_y, probs)

(array([1., 1., 1., 1., 1., 1., 1.]),
 array([1.        , 0.98888889, 0.96666667, 0.95555556, 0.93333333,
        0.92222222, 0.        ]),
 array([0.99999845, 0.99999934, 0.99999946, 0.9999995 , 0.9999999 ,
        1.        ], dtype=float32))

In [77]:
confusion_matrix(train_y, preds)

array([[75,  0],
       [ 0, 90]])

---

In [119]:
model.history.history

{}

In [120]:
results = model.evaluate(train_x, train_y, verbose=0)

In [121]:
results

[1.0197285238433551e-09, 1.0]